### Tools on LangChain

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# from langchain_google_vertexai import ChatVertexAI
from langchain_openai import ChatOpenAI

# llm = ChatVertexAI(model="gemini-2.0-flash-001")
llm = ChatOpenAI()
question = "how old is the US president?"

Modern LLMs hide the need to construct a prompt from the user, you can define your tools as schemas instead and pass them as a separate argument:

In [ ]:
search_tool = {
    "type": "function",
    "function": {
        "name": "google_search",
        "description": "Returns about common facts, fresh events and news from Google Search engine based on a query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "title": "search_query",
                    "description": "The search query to be sent to Google Search."
                }
            },
            "required": ["query"]
        }
    }
}

step1 = llm.invoke(question, tools=[search_tool])

As we can see, now our outputs contains a special part called tool_calls - again, there's no need for us to parse the output any more, it's all done by an LLM:

In [4]:
step1

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EQdNDckqQei4vVDahtEAgfky', 'function': {'arguments': '{"query":"current age of the US president"}', 'name': 'google_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 80, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3wg3OVvA7IOUco2MumHqfprfnYFx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--53f51bed-2b07-4551-95f6-6771f85650fc-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'current age of the US president'}, 'id': 'call_EQdNDckqQei4vVDahtEAgfky', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 19, 'total_t

In [5]:
print(step1.tool_calls)

[{'name': 'google_search', 'args': {'query': 'current age of the US president'}, 'id': 'call_EQdNDckqQei4vVDahtEAgfky', 'type': 'tool_call'}]


In [6]:
from langchain_core.messages import HumanMessage, ToolMessage

tool_result = ToolMessage(content="Donald Trump › Age 78 years June 14, 1946\n", tool_call_id=step1.tool_calls[0]["id"])
step2 = llm.invoke(
    [HumanMessage(content=question), step1, tool_result],
    tools=[search_tool]
)
assert len(step2.tool_calls) == 0

print(step2.content)

The current US President, Donald Trump, is 78 years old. He was born on June 14, 1946.


In [7]:
llm_with_tools = llm.bind(tools=[search_tool])
llm_with_tools.invoke(question)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pBerL40VtJ8YM2SY8ddmtpFv', 'function': {'arguments': '{"query":"age of US president"}', 'name': 'google_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 80, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C3wm3WxJNeuSQb7HF29niKTeKcKQy', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--147758d9-67ae-4d22-b720-b4a2daa645a9-0', tool_calls=[{'name': 'google_search', 'args': {'query': 'age of US president'}, 'id': 'call_pBerL40VtJ8YM2SY8ddmtpFv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 80, 'output_tokens': 17, 'total_tokens': 97, 'input_token